# Kết nối với Database

## Cấu hình SQL

In [21]:
import os
import pandas as pd
from sqlalchemy.types import NVARCHAR, Integer, Float, DateTime
from sqlalchemy.exc import IntegrityError
import unicodedata
import re
from sqlalchemy import create_engine, Integer, Float, DateTime, text
from sqlalchemy.dialects.mssql import NVARCHAR

In [22]:
# Cấu hình thông tin kết nối
server = r'DESKTOP-8H3JR7M\LUONGLOC'            # Ví dụ: 'localhost' hoặc 'SERVER_NAME\INSTANCE'
database = r'CaPhe'
username = r'sa'
password = r'123456'

In [23]:
# mssql+pyodbc://{sa}:{123456}@{DESKTOP-8H3JR7M\LUONGLOC}/{CaPheAnSinh}?driver=ODBC+Driver+17+for+SQL+Server&charset=utf8mb4

In [24]:
# Tạo chuỗi kết nối với UTF-8 encoding
conn_str = f"mssql+pyodbc://{username}:{password}@{server}/{database}?driver=ODBC+Driver+17+for+SQL+Server&charset=utf8mb4"
excel_path = r'D:\Báo cáo Thực tập\Data-Final.xlsx'

# import du lieu va rang buoc

In [25]:
import pandas as pd
import unicodedata
import re
from sqlalchemy import create_engine, text, inspect
from sqlalchemy.types import NVARCHAR, Integer, Float, DateTime
from sqlalchemy.exc import IntegrityError, SQLAlchemyError
import traceback # Để in chi tiết lỗi

# Tạo chuỗi kết nối với UTF-8 encoding
conn_str = f"mssql+pyodbc://{username}:{password}@{server}/{database}?driver=ODBC+Driver+17+for+SQL+Server&charset=utf8mb4"
excel_path = r'D:\Báo cáo Thực tập\Data-Final.xlsx'

# --- Hàm chuyển chuỗi Unicode có dấu thành không dấu ---
def remove_accents(input_str):
    """Chuyển chuỗi Unicode có dấu thành không dấu."""
    if not isinstance(input_str, str):
        return str(input_str)
    nfkd_form = unicodedata.normalize('NFKD', input_str)
    return ''.join([c for c in nfkd_form if not unicodedata.combining(c)])

# --- Hàm chuyển tên sheet/cột thành tên SQL hợp lệ ---
def normalize_sql_name(name, is_table=False):
    """Chuẩn hóa tên cho bảng hoặc cột SQL."""
    no_accent = remove_accents(name)
    normalized = re.sub(r'\s+', '_', no_accent)
    normalized = re.sub(r'[^a-zA-Z0-9_]', '', normalized)

    # Đảm bảo tên không bắt đầu bằng số
    if normalized and normalized[0].isdigit():
        prefix = 'T_' if is_table else 'C_'
        normalized = prefix + normalized

    # Đặt tên mặc định nếu thành rỗng sau khi chuẩn hóa
    if not normalized:
        normalized = "default_table" if is_table else "default_column"

    # Giới hạn độ dài tên (ví dụ: SQL Server tối đa 128)
    return normalized[:128]

# --- Hàm lấy kiểu dữ liệu SQL từ instance SQLAlchemy ---
def get_sql_type_string(dtype_instance):
    """Trả về chuỗi định nghĩa kiểu SQL từ instance SQLAlchemy."""
    if isinstance(dtype_instance, NVARCHAR):
        length = getattr(dtype_instance, 'length', 255)
        return f"NVARCHAR({length})"
    elif isinstance(dtype_instance, Integer):
        return "INT"
    elif isinstance(dtype_instance, Float):
        return "FLOAT"
    elif isinstance(dtype_instance, DateTime):
        return "DATETIME"
    else:
        # Kiểu mặc định nếu không nhận diện được
        return "NVARCHAR(255)"

# --- Hàm import dữ liệu theo chunk ---
def import_in_chunks(df, table_name, engine, dtype_mapping, chunk_size=1000):
    """Import DataFrame vào bảng SQL theo từng phần (chunk)."""
    if df.empty:
        print(f"[{table_name}] DataFrame trống, không có dữ liệu để import.")
        return False # Trả về False nếu không có gì để import

    try:
        # 1. Tạo/Thay thế cấu trúc bảng (chỉ tạo cột, chưa có PK, FK)
        df.head(0).to_sql(table_name, engine, if_exists="replace", index=False, dtype=dtype_mapping)
        print(f"[{table_name}] Đã tạo/thay thế cấu trúc bảng.")

        # 2. Import dữ liệu theo chunks
        total_rows = len(df)
        print(f"[{table_name}] Bắt đầu import {total_rows} hàng...")
        for i in range(0, total_rows, chunk_size):
            end = min(i + chunk_size, total_rows)
            chunk = df.iloc[i:end]
            try:
                # Dùng 'append' để thêm dữ liệu vào bảng đã có cấu trúc
                chunk.to_sql(table_name, engine, if_exists="append", index=False)
                # In tiến trình ít hơn để tránh quá nhiều output
                if (i + chunk_size) >= total_rows or i == 0:
                     print(f"[{table_name}] Đã import {end}/{total_rows} hàng.")
            except Exception as chunk_e:
                print(f"!!! Lỗi khi import chunk {i}-{end} vào bảng '{table_name}': {chunk_e}")
                # Quyết định: dừng lại hay tiếp tục? Hiện tại đang tiếp tục.
                # return False # Bỏ comment nếu muốn dừng hoàn toàn khi có lỗi chunk
        print(f"[{table_name}] Import dữ liệu hoàn tất.")
        return True # Trả về True nếu import thành công (ít nhất là cấu trúc)

    except Exception as e:
        print(f"!!! Lỗi nghiêm trọng trong quá trình import vào bảng '{table_name}': {e}")
        traceback.print_exc()
        return False

# --- Hàm tạo khóa chính ---
def set_primary_key(engine, table_name, pk_column_name, dtype_mapping):
    """Thiết lập cột chỉ định làm khóa chính cho bảng."""
    print(f"[{table_name}] Bắt đầu thiết lập PK cho cột '{pk_column_name}'.")
    pk_col_type_instance = dtype_mapping.get(pk_column_name)
    if not pk_col_type_instance:
        print(f"!!! Lỗi: Không tìm thấy kiểu dữ liệu đã mapping cho cột PK '{pk_column_name}' trong bảng '{table_name}'. Bỏ qua PK.")
        return False

    sql_type_for_pk = get_sql_type_string(pk_col_type_instance)
    pk_constraint_name = f"PK_{normalize_sql_name(table_name)}_{normalize_sql_name(pk_column_name)}"[:128] # Giới hạn độ dài tên constraint

    try:
        with engine.connect() as connection:
            with connection.begin(): # Bắt đầu transaction
                # 1. Đặt cột là NOT NULL
                print(f"[{table_name}] Đang đặt NOT NULL cho cột '{pk_column_name}'...")
                alter_col_sql = f'ALTER TABLE [{table_name}] ALTER COLUMN [{pk_column_name}] {sql_type_for_pk} NOT NULL'
                connection.execute(text(alter_col_sql))
                print(f"[{table_name}] Đã đặt cột '{pk_column_name}' là NOT NULL.")

                # 2. Thêm ràng buộc khóa chính
                print(f"[{table_name}] Đang thêm PRIMARY KEY '{pk_constraint_name}'...")
                add_pk_sql = f'ALTER TABLE [{table_name}] ADD CONSTRAINT [{pk_constraint_name}] PRIMARY KEY ([{pk_column_name}])'
                connection.execute(text(add_pk_sql))
                print(f"[{table_name}] Đã thiết lập khóa chính '{pk_constraint_name}' thành công.")
                return True # Trả về True nếu thành công

    except IntegrityError as ie:
         print(f"!!! Lỗi Integrity khi thiết lập PK cho '{table_name}'.'{pk_column_name}': {ie}")
         print(f"    Nguyên nhân có thể do giá trị trùng lặp trong cột '{pk_column_name}'.")
         return False
    except SQLAlchemyError as e:
        print(f"!!! Lỗi SQL khi thiết lập PK cho '{table_name}'.'{pk_column_name}': {e}")
        print("    Vui lòng kiểm tra dữ liệu cột PK và cấu trúc bảng.")
        # traceback.print_exc()
        return False
    except Exception as e:
        print(f"!!! Lỗi không xác định khi thiết lập PK cho '{table_name}'.'{pk_column_name}': {e}")
        traceback.print_exc()
        return False

# --- Hàm tạo khóa ngoại tự động ---
def create_foreign_keys(engine, tables_info):
    """
    Tự động tạo khóa ngoại dựa trên tên cột trùng khớp sau khi tất cả import hoàn tất.
    """
    print("\n--- Bắt đầu quá trình tạo khóa ngoại tự động ---")
    if not tables_info:
        print("Không có thông tin bảng nào để tạo khóa ngoại. Bỏ qua.")
        return

    # Lặp qua từng bảng xem nó như bảng "con" (có FK)
    for child_table, child_info in tables_info.items():
        print(f"\n[*] Đang kiểm tra bảng '{child_table}' cho các FK tiềm năng...")
        child_columns = child_info.get('columns', [])
        child_pk = child_info.get('pk_column')

        if not child_columns or not child_pk:
            print(f"    - Bỏ qua '{child_table}' do thiếu thông tin cột hoặc PK.")
            continue

        # Lặp qua từng bảng khác xem nó như bảng "cha" (được tham chiếu)
        for parent_table, parent_info in tables_info.items():
            if child_table == parent_table:
                continue # Không tạo FK tự tham chiếu trong logic này

            parent_pk = parent_info.get('pk_column')
            if not parent_pk:
                # Không cần in lỗi này vì bảng cha thiếu PK đã được xử lý khi nó là bảng con
                continue

            # Kiểm tra từng cột trong bảng con
            for child_column in child_columns:
                # --- Điều kiện quan trọng để tạo FK ---
                # 1. Tên cột con == Tên cột PK cha
                # 2. Cột con này KHÔNG PHẢI là PK của chính bảng con
                if child_column == parent_pk and child_column != child_pk:
                    print(f"    - Tìm thấy tiềm năng FK: '{child_table}'.[{child_column}] -> '{parent_table}'.[{parent_pk}]")

                    # Tạo ràng buộc khóa ngoại
                    fk_constraint_name = f"FK_{normalize_sql_name(child_table)}_{normalize_sql_name(child_column)}_{normalize_sql_name(parent_table)}"[:128]
                    try:
                        with engine.connect() as connection:
                            with connection.begin():
                                print(f"      -> Đang tạo ràng buộc '{fk_constraint_name}'...")
                                fk_sql = f"""
                                    ALTER TABLE [{child_table}]
                                    ADD CONSTRAINT [{fk_constraint_name}]
                                    FOREIGN KEY ([{child_column}])
                                    REFERENCES [{parent_table}]([{parent_pk}])
                                """
                                connection.execute(text(fk_sql))
                                print(f"      -> Tạo FK '{fk_constraint_name}' thành công.")
                    except IntegrityError as ie:
                         # Lỗi này thường xảy ra nếu dữ liệu vi phạm FK
                         print(f"!!! Lỗi Integrity khi tạo FK '{fk_constraint_name}': {ie}")
                         print(f"    Kiểm tra dữ liệu: Các giá trị trong '{child_table}'.[{child_column}] phải tồn tại trong '{parent_table}'.[{parent_pk}].")
                    except SQLAlchemyError as e:
                        # Lỗi SQL khác (kiểu không khớp, tên sai - ít khả năng hơn)
                        print(f"!!! Lỗi SQL khi tạo FK '{fk_constraint_name}': {e}")
                        # traceback.print_exc()
                    except Exception as e:
                        print(f"!!! Lỗi không xác định khi tạo FK '{fk_constraint_name}': {e}")
                        traceback.print_exc()
                    # Dù lỗi hay không, tiếp tục tìm FK tiềm năng khác
                    break # Đã tìm thấy FK cho cột này, chuyển sang cột tiếp theo của bảng con

    print("\n--- Hoàn tất quá trình tạo khóa ngoại tự động ---")

# --- Cấu hình ---
CHUNK_SIZE = 1000 # Số hàng import mỗi lần, điều chỉnh nếu cần

# --- Phần thực thi chính ---
if __name__ == "__main__":
    print("--- BẮT ĐẦU QUÁ TRÌNH IMPORT EXCEL VÀ TẠO SCHEMA ---")
    engine = None
    try:
        # 1. Tạo engine kết nối CSDL
        engine = create_engine(
            conn_str,
            fast_executemany=True  # <<< Nên dùng để tăng tốc độ với pyodbc
        )
        # Kiểm tra kết nối nhanh
        with engine.connect() as conn_test:
             print("Kết nối CSDL thành công.")

    except Exception as e:
        print(f"!!! Lỗi nghiêm trọng khi tạo engine hoặc kết nối CSDL: {e}")
        print(">>> Vui lòng kiểm tra chuỗi kết nối, thông tin đăng nhập, driver ODBC và trạng thái SQL Server.")
        traceback.print_exc()
        exit() # Thoát nếu không kết nối được CSDL

    all_imported_tables_info = {} # Dictionary lưu thông tin các bảng đã import và có PK thành công

    try:
        # 2. Đọc file Excel
        print(f"Đang đọc file Excel: {excel_path}")
        excel_data = pd.ExcelFile(excel_path)
        sheet_names = excel_data.sheet_names
        print(f"Tìm thấy {len(sheet_names)} sheet: {sheet_names}")

        # 3. Vòng lặp xử lý từng sheet
        for sheet_name in sheet_names:
            df = None # Reset df cho mỗi sheet
            # Chuẩn hóa tên bảng từ tên sheet
            table_name = normalize_sql_name(sheet_name, is_table=True)
            print(f"\n--- Xử lý sheet: '{sheet_name}' -> Bảng: '{table_name}' ---")

            try:
                # Đọc dữ liệu từ sheet hiện tại
                df = pd.read_excel(excel_path, sheet_name=sheet_name, engine='openpyxl')

                if df.empty:
                    print(f"Sheet '{sheet_name}' trống. Bỏ qua.")
                    continue

                # --- Chuẩn hóa tên cột ---
                original_columns = df.columns.tolist()
                new_columns = []
                counts = {}
                for col in original_columns:
                    col_str = str(col) # Đảm bảo là string
                    norm_col = normalize_sql_name(col_str, is_table=False) # Chuẩn hóa tên cột

                    # Xử lý trùng lặp *sau khi* chuẩn hóa
                    final_col_name = norm_col
                    if final_col_name in counts:
                        counts[final_col_name] += 1
                        final_col_name = f"{final_col_name}_{counts[final_col_name]}"
                    else:
                        counts[final_col_name] = 0
                    new_columns.append(final_col_name)

                df.columns = new_columns
                if original_columns != new_columns:
                    print(f"Đã chuẩn hóa/đổi tên cột. Cột mới: {new_columns}")

                # --- Xác định cột PK (cột đầu tiên sau chuẩn hóa) ---
                if not df.columns.empty:
                    pk_column_name = df.columns[0]
                    print(f"Cột PK dự kiến: '{pk_column_name}'")
                else:
                    print(f"Sheet '{sheet_name}' không có cột nào sau khi xử lý. Bỏ qua.")
                    continue

                # --- Xử lý giá trị NULL/NA ---
                # Loại bỏ hàng có giá trị NULL/NA ở cột PK
                rows_before_pk_dropna = len(df)
                df = df.dropna(subset=[pk_column_name])
                rows_dropped = rows_before_pk_dropna - len(df)
                if rows_dropped > 0:
                    print(f"Đã loại bỏ {rows_dropped} hàng có giá trị NULL/NA ở cột PK '{pk_column_name}'.")

                if df.empty:
                    print(f"Sheet '{sheet_name}' trống sau khi loại bỏ NULL ở cột PK. Bỏ qua.")
                    continue

                print(f"Số hàng hợp lệ còn lại: {len(df)}")

                # Xử lý NULL/NA ở các cột khác (fill bằng giá trị mặc định)
                # Lưu ý: Việc fillna có thể ảnh hưởng kiểu dữ liệu suy luận sau này
                for col in df.columns:
                    if col == pk_column_name: continue # Bỏ qua cột PK
                    if pd.api.types.is_numeric_dtype(df[col]):
                         # Quyết định: Fill số bằng 0 hay để NULL? Hiện tại để NULL (không fill)
                         # df[col] = df[col].fillna(0)
                         pass
                    elif pd.api.types.is_datetime64_any_dtype(df[col]):
                         # Để NULL cho datetime là phổ biến
                         pass
                    else: # String hoặc object
                        df[col] = df[col].fillna("") # Fill chuỗi rỗng

                # --- Xác định kiểu dữ liệu SQL và tạo mapping ---
                dtype_mapping = {}
                print("Đang xác định kiểu dữ liệu SQL cho các cột...")
                for col_name in df.columns:
                    dtype = df[col_name].dtype
                    try:
                        if pd.api.types.is_string_dtype(dtype) or pd.api.types.is_object_dtype(dtype):
                            # Tính max_len cẩn thận hơn
                            non_na_strings = df[col_name].dropna().astype(str)
                            if not non_na_strings.empty:
                                max_len = non_na_strings.str.len().max()
                            else:
                                max_len = 0 # Nếu cột toàn NA (sau fillna có thể là "")

                            effective_len = min(max(50, int(max_len) + 1), 4000) # Ít nhất 50, nhiều nhất 4000
                            dtype_mapping[col_name] = NVARCHAR(effective_len)
                        elif pd.api.types.is_integer_dtype(dtype):
                            dtype_mapping[col_name] = Integer()
                        elif pd.api.types.is_float_dtype(dtype):
                            dtype_mapping[col_name] = Float()
                        elif pd.api.types.is_datetime64_any_dtype(dtype):
                             # Chuyển đổi sang datetime nếu chưa phải để to_sql xử lý đúng
                             try:
                                 df[col_name] = pd.to_datetime(df[col_name], errors='coerce')
                             except Exception:
                                 print(f"Cảnh báo: Không thể chuyển cột '{col_name}' sang datetime. Giữ nguyên.")
                             dtype_mapping[col_name] = DateTime()
                        elif pd.api.types.is_bool_dtype(dtype):
                            dtype_mapping[col_name] = Integer() # SQL Server thường dùng INT/BIT cho boolean
                        else:
                            print(f"Cảnh báo: Kiểu Pandas không xác định '{dtype}' cho cột '{col_name}', dùng NVARCHAR(255).")
                            dtype_mapping[col_name] = NVARCHAR(255)
                    except Exception as map_err:
                         print(f"!!! Lỗi khi xác định kiểu cho cột '{col_name}': {map_err}. Dùng NVARCHAR(255).")
                         dtype_mapping[col_name] = NVARCHAR(255)
                # print(f"Mapping kiểu dữ liệu: {dtype_mapping}") # Bỏ comment để xem chi tiết mapping

                # --- Import dữ liệu vào CSDL ---
                import_success = import_in_chunks(df, table_name, engine, dtype_mapping, CHUNK_SIZE)

                # --- Thiết lập khóa chính ---
                if import_success:
                    pk_success = set_primary_key(engine, table_name, pk_column_name, dtype_mapping)
                    if pk_success:
                        # Chỉ lưu thông tin nếu cả import và PK thành công
                        all_imported_tables_info[table_name] = {
                            'columns': df.columns.tolist(),
                            'pk_column': pk_column_name
                        }
                        print(f"[{table_name}] Đã lưu thông tin để tạo FK sau này.")
                    else:
                        print(f"[{table_name}] Không thể thiết lập PK, bảng này sẽ không được dùng để tạo FK.")
                else:
                    print(f"[{table_name}] Import dữ liệu thất bại, bỏ qua PK và FK.")

            except Exception as sheet_err:
                print(f"!!! Lỗi không xác định khi xử lý sheet '{sheet_name}': {sheet_err}")
                traceback.print_exc()
                # Tiếp tục xử lý sheet tiếp theo

        # --- Kết thúc vòng lặp qua các sheet ---

        # 4. Tạo khóa ngoại (sau khi tất cả đã được import và có PK)
        create_foreign_keys(engine, all_imported_tables_info)

    except FileNotFoundError:
        print(f"!!! Lỗi: Không tìm thấy file Excel tại đường dẫn: {excel_path}")
    except SQLAlchemyError as db_err:
         print(f"!!! Lỗi SQLAlchemy trong quá trình xử lý chính: {db_err}")
         traceback.print_exc()
    except Exception as general_err:
        print(f"!!! Lỗi nghiêm trọng không xác định xảy ra: {general_err}")
        traceback.print_exc()
    finally:
        # Đóng kết nối nếu engine được tạo thành công
        if engine:
            engine.dispose()
            print("\nĐã đóng kết nối CSDL.")

    print("\n--- HOÀN TẤT TOÀN BỘ QUÁ TRÌNH ---")

--- BẮT ĐẦU QUÁ TRÌNH IMPORT EXCEL VÀ TẠO SCHEMA ---
Kết nối CSDL thành công.
Đang đọc file Excel: D:\Báo cáo Thực tập\Data-Final.xlsx
Tìm thấy 10 sheet: ['KhachHang', 'LoaiKhachHang', 'NhaCungCap', 'ThucDon', 'TonKho', 'CongThuc', 'HinhAnh', 'LoaiHangHoa', 'HoaDonBanHang', 'ChiTietHoaDon']

--- Xử lý sheet: 'KhachHang' -> Bảng: 'KhachHang' ---
Cột PK dự kiến: 'MaKhachHang'
Số hàng hợp lệ còn lại: 2000
Đang xác định kiểu dữ liệu SQL cho các cột...
!!! Lỗi nghiêm trọng trong quá trình import vào bảng 'KhachHang': (pyodbc.ProgrammingError) ('42000', "[42000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Could not drop object 'KhachHang' because it is referenced by a FOREIGN KEY constraint. (3726) (SQLExecDirectW)")
[SQL: 
DROP TABLE [KhachHang]]
(Background on this error at: https://sqlalche.me/e/20/f405)
[KhachHang] Import dữ liệu thất bại, bỏ qua PK và FK.

--- Xử lý sheet: 'LoaiKhachHang' -> Bảng: 'LoaiKhachHang' ---


Traceback (most recent call last):
  File "d:\App\Miniconda\envs\bctt\lib\site-packages\sqlalchemy\engine\base.py", line 1964, in _exec_single_context
    self.dialect.do_execute(
  File "d:\App\Miniconda\envs\bctt\lib\site-packages\sqlalchemy\engine\default.py", line 942, in do_execute
    cursor.execute(statement, parameters)
pyodbc.ProgrammingError: ('42000', "[42000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Could not drop object 'KhachHang' because it is referenced by a FOREIGN KEY constraint. (3726) (SQLExecDirectW)")

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "C:\Users\TechCare\AppData\Local\Temp\ipykernel_7048\1140237568.py", line 65, in import_in_chunks
    df.head(0).to_sql(table_name, engine, if_exists="replace", index=False, dtype=dtype_mapping)
  File "d:\App\Miniconda\envs\bctt\lib\site-packages\pandas\util\_decorators.py", line 333, in wrapper
    return func(*args, **kwargs)
  File "d:\App

Cột PK dự kiến: 'MaLoaiKH'
Số hàng hợp lệ còn lại: 3
Đang xác định kiểu dữ liệu SQL cho các cột...
!!! Lỗi nghiêm trọng trong quá trình import vào bảng 'LoaiKhachHang': (pyodbc.ProgrammingError) ('42000', "[42000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Could not drop object 'LoaiKhachHang' because it is referenced by a FOREIGN KEY constraint. (3726) (SQLExecDirectW)")
[SQL: 
DROP TABLE [LoaiKhachHang]]
(Background on this error at: https://sqlalche.me/e/20/f405)
[LoaiKhachHang] Import dữ liệu thất bại, bỏ qua PK và FK.

--- Xử lý sheet: 'NhaCungCap' -> Bảng: 'NhaCungCap' ---


Traceback (most recent call last):
  File "d:\App\Miniconda\envs\bctt\lib\site-packages\sqlalchemy\engine\base.py", line 1964, in _exec_single_context
    self.dialect.do_execute(
  File "d:\App\Miniconda\envs\bctt\lib\site-packages\sqlalchemy\engine\default.py", line 942, in do_execute
    cursor.execute(statement, parameters)
pyodbc.ProgrammingError: ('42000', "[42000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Could not drop object 'LoaiKhachHang' because it is referenced by a FOREIGN KEY constraint. (3726) (SQLExecDirectW)")

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "C:\Users\TechCare\AppData\Local\Temp\ipykernel_7048\1140237568.py", line 65, in import_in_chunks
    df.head(0).to_sql(table_name, engine, if_exists="replace", index=False, dtype=dtype_mapping)
  File "d:\App\Miniconda\envs\bctt\lib\site-packages\pandas\util\_decorators.py", line 333, in wrapper
    return func(*args, **kwargs)
  File "d:

Cột PK dự kiến: 'MaNCC'
Số hàng hợp lệ còn lại: 6
Đang xác định kiểu dữ liệu SQL cho các cột...
!!! Lỗi nghiêm trọng trong quá trình import vào bảng 'NhaCungCap': (pyodbc.ProgrammingError) ('42000', "[42000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Could not drop object 'NhaCungCap' because it is referenced by a FOREIGN KEY constraint. (3726) (SQLExecDirectW)")
[SQL: 
DROP TABLE [NhaCungCap]]
(Background on this error at: https://sqlalche.me/e/20/f405)
[NhaCungCap] Import dữ liệu thất bại, bỏ qua PK và FK.

--- Xử lý sheet: 'ThucDon' -> Bảng: 'ThucDon' ---


...

Cột PK dự kiến: 'MaNL'
Số hàng hợp lệ còn lại: 53
Đang xác định kiểu dữ liệu SQL cho các cột...
[TonKho] Đã tạo/thay thế cấu trúc bảng.
[TonKho] Bắt đầu import 53 hàng...
[TonKho] Đã import 53/53 hàng.
[TonKho] Import dữ liệu hoàn tất.
[TonKho] Bắt đầu thiết lập PK cho cột 'MaNL'.
[TonKho] Đang đặt NOT NULL cho cột 'MaNL'...
[TonKho] Đã đặt cột 'MaNL' là NOT NULL.
[TonKho] Đang thêm PRIMARY KEY 'PK_TonKho_MaNL'...
[TonKho] Đã thiết lập khóa chính 'PK_TonKho_MaNL' thành công.
[TonKho] Đã lưu thông tin để tạo FK sau này.

--- Xử lý sheet: 'CongThuc' -> Bảng: 'CongThuc' ---
Cột PK dự kiến: 'ID'
Số hàng hợp lệ còn lại: 90
Đang xác định kiểu dữ liệu SQL cho các cột...
[CongThuc] Đã tạo/thay thế cấu trúc bảng.
[CongThuc] Bắt đầu import 90 hàng...
[CongThuc] Đã import 90/90 hàng.
[CongThuc] Import dữ liệu hoàn tất.
[CongThuc] Bắt đầu thiết lập PK cho cột 'ID'.
[CongThuc] Đang đặt NOT NULL cho cột 'ID'...
[CongThuc] Đã đặt cột 'ID' là NOT NULL.
[CongThuc] Đang thêm PRIMARY KEY 'PK_CongThuc_ID'

Traceback (most recent call last):
  File "d:\App\Miniconda\envs\bctt\lib\site-packages\sqlalchemy\engine\base.py", line 1964, in _exec_single_context
    self.dialect.do_execute(
  File "d:\App\Miniconda\envs\bctt\lib\site-packages\sqlalchemy\engine\default.py", line 942, in do_execute
    cursor.execute(statement, parameters)
pyodbc.ProgrammingError: ('42000', "[42000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Could not drop object 'LoaiHangHoa' because it is referenced by a FOREIGN KEY constraint. (3726) (SQLExecDirectW)")

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "C:\Users\TechCare\AppData\Local\Temp\ipykernel_7048\1140237568.py", line 65, in import_in_chunks
    df.head(0).to_sql(table_name, engine, if_exists="replace", index=False, dtype=dtype_mapping)
  File "d:\App\Miniconda\envs\bctt\lib\site-packages\pandas\util\_decorators.py", line 333, in wrapper
    return func(*args, **kwargs)
  File "d:\A

Cột PK dự kiến: 'MaHD'
Số hàng hợp lệ còn lại: 315
Đang xác định kiểu dữ liệu SQL cho các cột...
!!! Lỗi nghiêm trọng trong quá trình import vào bảng 'HoaDonBanHang': (pyodbc.ProgrammingError) ('42000', "[42000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Could not drop object 'HoaDonBanHang' because it is referenced by a FOREIGN KEY constraint. (3726) (SQLExecDirectW)")
[SQL: 
DROP TABLE [HoaDonBanHang]]
(Background on this error at: https://sqlalche.me/e/20/f405)
[HoaDonBanHang] Import dữ liệu thất bại, bỏ qua PK và FK.

--- Xử lý sheet: 'ChiTietHoaDon' -> Bảng: 'ChiTietHoaDon' ---


Traceback (most recent call last):
  File "d:\App\Miniconda\envs\bctt\lib\site-packages\sqlalchemy\engine\base.py", line 1964, in _exec_single_context
    self.dialect.do_execute(
  File "d:\App\Miniconda\envs\bctt\lib\site-packages\sqlalchemy\engine\default.py", line 942, in do_execute
    cursor.execute(statement, parameters)
pyodbc.ProgrammingError: ('42000', "[42000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Could not drop object 'HoaDonBanHang' because it is referenced by a FOREIGN KEY constraint. (3726) (SQLExecDirectW)")

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "C:\Users\TechCare\AppData\Local\Temp\ipykernel_7048\1140237568.py", line 65, in import_in_chunks
    df.head(0).to_sql(table_name, engine, if_exists="replace", index=False, dtype=dtype_mapping)
  File "d:\App\Miniconda\envs\bctt\lib\site-packages\pandas\util\_decorators.py", line 333, in wrapper
    return func(*args, **kwargs)
  File "d:

Cột PK dự kiến: 'ID'
Số hàng hợp lệ còn lại: 100
Đang xác định kiểu dữ liệu SQL cho các cột...
[ChiTietHoaDon] Đã tạo/thay thế cấu trúc bảng.
[ChiTietHoaDon] Bắt đầu import 100 hàng...
[ChiTietHoaDon] Đã import 100/100 hàng.
[ChiTietHoaDon] Import dữ liệu hoàn tất.
[ChiTietHoaDon] Bắt đầu thiết lập PK cho cột 'ID'.
[ChiTietHoaDon] Đang đặt NOT NULL cho cột 'ID'...
[ChiTietHoaDon] Đã đặt cột 'ID' là NOT NULL.
[ChiTietHoaDon] Đang thêm PRIMARY KEY 'PK_ChiTietHoaDon_ID'...
[ChiTietHoaDon] Đã thiết lập khóa chính 'PK_ChiTietHoaDon_ID' thành công.
[ChiTietHoaDon] Đã lưu thông tin để tạo FK sau này.

--- Bắt đầu quá trình tạo khóa ngoại tự động ---

[*] Đang kiểm tra bảng 'TonKho' cho các FK tiềm năng...
    - Tìm thấy tiềm năng FK: 'TonKho'.[MaHH] -> 'HinhAnh'.[MaHH]
      -> Đang tạo ràng buộc 'FK_TonKho_MaHH_HinhAnh'...
!!! Lỗi Integrity khi tạo FK 'FK_TonKho_MaHH_HinhAnh': (pyodbc.IntegrityError) ('23000', '[23000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]The ALTER TABLE sta